<a href="https://colab.research.google.com/github/LoPA607/IE643_project/blob/main/vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q timm==0.9.2 torch torchvision pillow tqdm

In [ ]:
import os
import random
from PIL import Image
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from torchvision import transforms
from torchvision.utils import save_image
import math

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_SIZE = 224                 # ViT base patch16 standard size
VIT_NAME = 'vit_base_patch16_224'
EXTRACT_LAYERS = [2, 5, 8, 11] # layer indices to extract features from
NUM_STEPS = 50
LR = 0.002
STYLE_WEIGHT = 1e9
CONTENT_WEIGHT = 0.000001
TV_WEIGHT = 1
JITTER_MAX = 3              # max pixel jitter to apply each step (helps remove seams)
MULTI_SCALE = [1.0, 0.75, 0.5] # optimize at these image scales (coarse->fine)
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
def load_image(path, size=IMG_SIZE):
    img = Image.open(path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])
    return transform(img).unsqueeze(0).to(DEVICE)

def denorm_tensor(tensor):
    # input in [0,1], returns clamped [0,1]
    return tensor.clamp(0., 1.)

def save_out(tensor, path):
    t = denorm_tensor(t.detach().cpu())
    save_image(t, path)

In [ ]:
class ViTTokenExtractor:
    def __init__(self, model, layers):
        """
        model: timm ViT model
        layers: list of transformer block indices to hook outputs from
        """
        self.model = model
        self.layers = layers
        self._hooks = []
        self.outputs = {}
        self._register_hooks()

    def _hook_fn(self, idx):
        def hook(module, input, output):
            # output shape (B, N+1, D) for timm ViT blocks; keep all tokens
            self.outputs[idx] = output
        return hook

    def _register_hooks(self):
        # timm ViT typically has model.blocks
        for idx in self.layers:
            block = self.model.blocks[idx]
            h = block.register_forward_hook(self._hook_fn(idx))
            self._hooks.append(h)

    def remove(self):
        for h in self._hooks:
            h.remove()

    @torch.no_grad()
    def features(self, x):
        """Run forward_features and return dict {layer_idx: tokens (B, N, D) excluding cls}"""
        self.outputs.clear()
        _ = self.model.forward_features(x)
        feats = {}
        for k, v in self.outputs.items():
            # drop cls token at index 0 -> get patch tokens: (B, N, D)
            tokens = v[:, 1:, :].detach()  # (B, N, D)
            feats[k] = tokens
        return feats

In [ ]:
def gram_tokens(tokens):
    """
    tokens: (B, N, D)
    returns gram: (B, D, D)
    """
    B, N, D = tokens.shape
    t = tokens.transpose(1, 2)  # (B, D, N)
    g = torch.bmm(t, tokens) / (D * N)
    return g

In [ ]:
def total_variation_loss(x):
    # x: (B, C, H, W), values in [0,1]
    diff_x = x[:, :, :, 1:] - x[:, :, :, :-1]
    diff_y = x[:, :, 1:, :] - x[:, :, :-1, :]
    return torch.mean(torch.abs(diff_x)) + torch.mean(torch.abs(diff_y))

In [ ]:
def random_jitter(img, max_jitter):
    """Random circular shift (wrap-around) by up to +/- max_jitter pixels in x and y."""
    if max_jitter <= 0:
        return img, (0, 0)
    B, C, H, W = img.shape
    dx = random.randint(-max_jitter, max_jitter)
    dy = random.randint(-max_jitter, max_jitter)
    # roll performs wrap-around; helps avoid black borders
    img_j = torch.roll(img, shifts=(dy, dx), dims=(2, 3))
    return img_j, (dx, dy)

def undo_jitter(img, shift):
    dx, dy = shift
    return torch.roll(img, shifts=(-dy, -dx), dims=(2, 3))

In [ ]:
def vit_style_transfer(content_path, style_path, out_prefix='vit_stylized',
                       steps=NUM_STEPS, lr=LR,
                       style_w=STYLE_WEIGHT, content_w=CONTENT_WEIGHT, tv_w=TV_WEIGHT):
    # Load ViT
    print("Loading ViT model:", VIT_NAME)
    vit = timm.create_model(VIT_NAME, pretrained=True).to(DEVICE).eval()
    extractor = ViTTokenExtractor(vit, EXTRACT_LAYERS)

    # Load images (we will rescale for multi-scale)
    content_full = Image.open(content_path).convert('RGB')
    style_full = Image.open(style_path).convert('RGB')

    # Precompute style features at base scale (ViT expects specific size)
    style_tensor_base = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)), transforms.ToTensor()])(style_full).unsqueeze(0).to(DEVICE)
    style_tensor_base = normalize_imagenet(style_tensor_base)
    style_feats_base = extractor.features(style_tensor_base)

    # define initial image: start from content (resized at base scale)
    base_size = IMG_SIZE
    content_tensor = transforms.Compose([transforms.Resize((base_size, base_size)), transforms.ToTensor()])(content_full).unsqueeze(0).to(DEVICE)
    content_tensor = normalize_imagenet(content_tensor)

    # target: optimize pixels starting from content
    target = content_tensor.clone().requires_grad_(True)

    optimizer = optim.Adam([target], lr=lr)

    # helper: feature extract for content at base scale
    content_feats_base = extractor.features(content_tensor)

    print("Starting optimization...")
    for step in tqdm(range(1, steps + 1)):
        optimizer.zero_grad()

        # Multi-scale losses: compute at each scale (coarse to fine)
        total_style_loss = 0.0
        total_content_loss = 0.0

        for scale in MULTI_SCALE:
            # resize target to current scale
            s = max(32, int(IMG_SIZE * scale))
            target_resized = nn.functional.interpolate(target, size=(s, s), mode='bilinear', align_corners=False)

            # apply random jitter to break patch alignment
            target_jittered, shift = random_jitter(target_resized, max_jitter=JITTER_MAX)

            # extract features (tokens) for target_jittered
            # NOTE: ViT feature extraction expects IMG_SIZE input, so resize here before extracting
            target_for_feats = nn.functional.interpolate(target_jittered, size=(IMG_SIZE, IMG_SIZE), mode='bilinear', align_corners=False)
            feats_t = extractor.features(target_for_feats)  # dict layer->(B,N,D)


            # content loss only at base scale (or we can compute across scales)
            if math.isclose(scale, 1.0):
                for k in content_feats_base:
                    cf = content_feats_base[k]
                    tf = feats_t[k]
                    total_content_loss += nn.functional.mse_loss(tf, cf)

            # style loss: compare Gram matrices at this scale between target and precomputed style
            # NOTE: Style feats were precomputed at base scale, so use those
            style_feats = style_feats_base # Use base scale style features
            for k in style_feats:
                gram_t = gram_tokens(feats_t[k])
                gram_s = gram_tokens(style_feats[k])
                total_style_loss += nn.functional.mse_loss(gram_t, gram_s)

        # normalize by number of scales and layers to keep magnitudes stable
        n_scales = len(MULTI_SCALE)
        n_layers = len(EXTRACT_LAYERS)
        total_style_loss = total_style_loss / (n_scales * n_layers)
        total_content_loss = total_content_loss / max(1.0, n_layers)  # content computed only at base

        # total variation smoothing
        tv_loss = total_variation_loss(target)

        loss = content_w * total_content_loss + style_w * total_style_loss + tv_w * tv_loss
        loss.backward()
        optimizer.step()

        # ensure target remains in [0,1] before normalization
        with torch.no_grad():
            # undo normalization clamping: target is in normalized domain; map to 0..1 then renormalize
            t = denormalize_imagenet(target)
            t = t.clamp(0.0, 1.0)
            target.data.copy_(normalize_imagenet(t))

        if step % 50 == 0 or step == steps:
            # save intermediate result (denormalize)
            out_img = denormalize_imagenet(target).detach().cpu().squeeze(0)
            save_image(out_img.clamp(0,1), f"{out_prefix}_step{step}.png")
            print(f"Step {step}: loss={loss.item():.4f} (style={total_style_loss.item():.4f}, content={total_content_loss.item():.4f}, tv={tv_loss.item():.6f})")

    extractor.remove()
    final_img = denormalize_imagenet(target).detach().cpu().squeeze(0)
    save_image(final_img.clamp(0,1), f"{out_prefix}_final.png")
    print("Saved final:", f"{out_prefix}_final.png")
    return f"{out_prefix}_final.png"

In [ ]:
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406], device=DEVICE).view(1,3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225], device=DEVICE).view(1,3,1,1)

def normalize_imagenet(x):
    return (x - IMAGENET_MEAN) / IMAGENET_STD

def denormalize_imagenet(x):
    return x * IMAGENET_STD + IMAGENET_MEAN

# -----------------------
# Small helpers
# -----------------------
from torchvision.utils import save_image
from IPython.display import display
import matplotlib.pyplot as plt

def display_image(path, title):
    img = Image.open(path)
    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()


# -----------------------
# Run example (update paths)
# -----------------------
if __name__ == "__main__":
    content_path = "content2.jpg"   # change to your content path
    style_path = "style3.jpeg"      # change to your style path
    out = vit_style_transfer(content_path, style_path, out_prefix='vit_hybrid', steps=800)
    print("Output saved to:", out)

    # Display images
    display_image(content_path, "Content Image")
    display_image(style_path, "Style Image")
    display_image(out, "Stylized Image")